In [ ]:
import os
import json

# Load setting from Json outside of project.
f = open('../../env/ai.json')
settingsJson = json.load(f)
del f

for key in settingsJson:
    os.environ[key] = settingsJson[key]
    
del settingsJson

In [ ]:
from langchain.document_loaders import PyPDFLoader 
from langchain.embeddings import OpenAIEmbeddings 
from langchain.vectorstores import Chroma 
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.agents import Tool

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader('./data/pdfs')
docs = loader.load()

In [ ]:
#print(docs)

In [ ]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(docs, embedding=embeddings) #,  persist_directory=".")

In [ ]:
# Search Tool  - https://serpapi.com/blog/filtering-google-search-and-google-news-results/

#!pip install google-search-results
from langchain.utilities import SerpAPIWrapper
search = SerpAPIWrapper()

def news_search(query: str):
    # https://serpapi.com/blog/filtering-google-search-and-google-news-results/
    # Past Hour: tbs-qdr:h
    # Past week: tbs-qdr:w
    # tbs=sbd:1
    # News:     tbm=nws
    # tbm=nws q:
    result = search.run(f"""" 
                        {query}
                        """)
    return result

news_search_tool = Tool(
    name='Public search tool', 
    func= news_search,    
    description="""
        Useful for searching for latest news. 
        """
)

# result = news_search("Cape Town")
# print(result)

Cape Town is a port city on South Africa’s southwest coast, on a peninsula beneath the imposing Table Mountain. Slowly rotating cable cars climb to the mountain’s flat top, from which there are sweeping views of the city, the busy harbor and boats heading for Robben Island, the notorious prison that once held Nelson Mandela, which is now a living museum. ― Google


In [ ]:
questions = """ 

Question: What do you specialise in?
Question: Can you tell me the latest news about Kenya?

"""

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent

tools = [news_search_tool]

#llm = OpenAI(temperature=0)
memory = ConversationBufferWindowMemory(memory_key='chat_history', k=3, return_messages=True)

# create our agent
conversational_agent = initialize_agent(agent='chat-conversational-react-description', tools=tools, llm=ChatOpenAI(temperature=0.9, model_name='gpt-3.5-turbo'), 
    verbose=True, max_iterations=3, early_stopping_method='generate', memory=memory)
# conversational_agent = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.8), vectordb.as_retriever(), memory=memory 
#                                                              ,verbose=True
#                                                              , tools=tools
#                                                              )

question = """
You are a helpful and empathetic financial expert explaining financial documents to people, please explain your answer to me as if I am a 10 year old. 
Think through and explain how you go to your answer. 

Question: Can you tell me the latest news about Kenya relates to authorised dealers wanting to send money to Kenya?

"""
answer = conversational_agent(question)
#print(answer)
#print(answer['answer'])



> Entering new AgentExecutor chain...
{
    "action": "Public search tool",
    "action_input": "latest news about Kenya authorised dealers sending money to Kenya"
}
Observation: What would you like to see featured on the new Kenyan currency? 10/02/2012, Reduced Clearing cycle for Cheques. 17/04/2012, Establishment of the Financial ...
Thought:{
    "action": "Final Answer",
    "action_input": "The latest news about Kenya related to authorized dealers wanting to send money to Kenya is not available. It seems that there is no recent news specifically addressing this topic." 
}

> Finished chain.


In [ ]:
conversational_agent("Question: Tell me about authorised dealers, what are they?")




> Entering new AgentExecutor chain...
{
    "action": "Public search tool",
    "action_input": "What are authorised dealers in finance?"
}
Observation: An authorized forex dealer is a type of financial institution that has received authorization from a relevant regulatory body to act as a dealer involved in the trading of foreign currencies.
Thought:{
    "action": "Final Answer",
    "action_input": "An authorised dealer in finance is a type of financial institution that has received authorization from a relevant regulatory body to act as a dealer involved in the trading of foreign currencies."
}

> Finished chain.


{'input': 'Question: Tell me about authorised dealers, what are they?',
 'chat_history': [HumanMessage(content='\nYou are a helpful and empathetic financial expert explaining financial documents to people, please explain your answer to me as if I am a 10 year old. \nThink through and explain how you go to your answer. \n\nQuestion: Can you tell me the latest news about Kenya relates to authorised dealers wanting to send money to Kenya?\n\n', additional_kwargs={}, example=False),
  AIMessage(content='The latest news about authorised dealers wanting to send money to Kenya is that you can send money securely from your phone to loved ones in Kenya using the Sendwave app. You can send money to M-PESA mobile wallets and select banks with no fees.', additional_kwargs={}, example=False)],
 'output': 'An authorised dealer in finance is a type of financial institution that has received authorization from a relevant regulatory body to act as a dealer involved in the trading of foreign currencies.